In [1]:
import pandas as pd
import asyncio
import matplotlib.pyplot as plt
import tqdm
from openai import AsyncAzureOpenAI
from sklearn.metrics import root_mean_squared_error

In [2]:
# Mati
API_KEY = "Fopj2HvyGgUx60yC311eSwgFUCi0nDMswQzGWB3Dz7BuwH7z2MJ3JQQJ99BFACHYHv6XJ3w3AAABACOGaGMT"
ENDPOINT = "https://gpt4o-mini-mati.openai.azure.com/"
DEPLOYMENT = "gpt-4o-mini"

In [3]:
from tqdm.notebook import tqdm

import os
import pandas as pd
from tqdm.asyncio import tqdm  # versión async-friendly de tqdm
from openai import AsyncAzureOpenAI

# ── Configuración del cliente Azure ────────────────────────────────────────────
client = AsyncAzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=ENDPOINT,   # p.ej. "https://<recurso>.openai.azure.com/"
    api_key=API_KEY
)

# ── Función: resume y guarda ──────────────────────────────────────────────────
async def resumir_CSV(
    cv_list,
    overwrite=True,
    temperatura=0.4
):
    # ── Prompt base (f-string se rellenará dentro del for)
    prompt_base = """
You will receive CV information fields with this format:

Career Objective: Big data analytics working and database warehouse manager with robust experience in handling all kinds of data. I have also used multiple cloud infrastructure services and am well acquainted with them. Currently in search of role that offers more of development.. Skills: ['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapreduce', 'Spark', 'Java', 'Machine Learning', 'Cloud', 'Hdfs', 'YARN', 'Core Java', 'Data Science', 'C++', 'Data Structures', 'DBMS', 'RDBMS', 'Informatica', 'Talend', 'Amazon Redshift', 'Microsoft Azure']. Institution: ['The Amity School of Engineering & Technology (ASET), Noida']. Degree: ['B.Tech']. Results: ['N/A']. Result Type: [None]. Field of Study: ['Electronics']. Companies: ['Coca-COla']. Job Skills: [['Big Data']]. Positions: ['Big Data Analyst']. Responsibilities: [Technical Support, Troubleshooting, Collaboration, Documentation, System Monitoring, Software Deployment, Training & Mentorship, Industry Trends, Field Visits]. Activity Type: null. Organizations: null. Roles: null. Languages: null. Proficiency: null. Certifications From: null. Certification Skills: null

Your task is to extract a cleaned and structured version of the resume that removes any personal contact information (name, email, phone, address, LinkedIn, etc.)
- Do NOT summarize or omit key content.
- Instead, preserve as much of the original job-related information as possible.
- Reorganize and rephrase disconnected items into full sentences with proper structure and connectors (e.g., “The candidate worked at...”, “They were responsible for...”, “Their skills include...”).
- Do NOT include any personal identifiers or contact information.
- Imagine you are preparing the resume for analysis by an AI model – you want to keep the full context but make it more readable.
You may use the following fields **only if present** in the text:
    - Career Objective: Brief description of career goals and desired role
    - Skills: List of technical and soft skills
    - Institution: Educational institutions attended
    - Degree: Academic degrees obtained
    - Results: Academic results (GPA, grades, honors)
    - Result Type: Type of academic result (GPA, Percentage, N/A)
    - Field of Study: Areas of study or specialization
    - Companies: Companies where the candidate has worked
    - Job Skills: Skills used in each job position
    - Positions: Job titles/roles held
    - Responsibilities: Key responsibilities in each role
    - Activity Type: Types of activities/involvement
    - Organizations: Organizations involved with
    - Roles: Roles in organizations
    - Languages: Languages spoken
    - Proficiency: Language proficiency levels
    - Certifications From: Certification providers
    - Certification Skills: Skills from certifications

Return your answer strictly CSV format

This is an example of output:
The candidate is a highly analytical and detail-oriented professional with extensive experience in financial statement preparation, quality assurance, compliance, auditing, customer service, and regulatory requirements. They possess thorough knowledge of financial reporting, cash and accrual basis accounting, fund basis reporting, GAAP, and other accounting standards. The candidate adapts quickly to new software and systems and is proficient in QuickBooks, MAS, ADP, Condo Manager, Adobe, and all Microsoft applications. They excel in problem-solving and strategic planning skills.

The candidate holds a Bachelor of Science in Accounting and a Bachelor of Science in Business Administration Finance, both obtained in May 2010 from the University of North Carolina.

In their role as a Staff Accountant from January 2014 to October 2014, they performed all month-end functions to produce financial statements, which included reconciling operating and payroll bank accounts, recording monthly sales, payroll journals, accrued payroll, sales tax, and inventory adjustments. They entered all payables and receivables for seven franchise locations, worked with vendors to resolve issues, and ensured that all necessary franchise documents were current. The candidate collaborated directly with the controller on year-end reviews and maintained a weekly spreadsheet on collected franchise royalties and advertising fees, which was utilized for year-end audits. Additionally, they managed a company that owned and rented out twelve commercial properties, negotiating rent rates, writing leases, collecting rent, managing cash flow, and addressing tenant complaints. They also managed a small farming entity, overseeing cash flow, payables, and payroll journals, and filed the North Carolina Annual Report for all companies managed by Cary Keisler Inc.

From March 2013 to January 2014, the candidate continued as a Staff Accountant, where they collected daily deposits, interacted with customers regarding outstanding balances, and handled all payable entries for various companies. They reconciled monthly purchasing American Express credit cards and facilitated the transition of the purchasing manager during a turnover period. They were responsible for ordering materials for the production floor and collaborated with the VP of Finance to develop unit valuation data for future job costing.

As a Staff Accountant from April 2012 to March 2013, the candidate led the reconciliation of the Balance Sheet, researched transaction discrepancies, and ensured accurate billing information for projects. They designed Excel spreadsheets for prepaid employee benefits reconciliation and managed the custodial bookkeeping of fixed assets for depreciation purposes. They also prepared monthly Sales and Use Tax.

From July 2010 to August 2011, the candidate served as a Financial Management Partner, where they identified and resolved subsidiary ledger discrepancies, performed monthly bank reconciliations, and provided support to clients and employees on financial reporting issues. They trained new accounting department employees and coordinated lien and foreclosure preparations. The candidate conducted constant analysis of clients' interim financial reports, prepared variance reports, and assisted in annual budget preparations and audits.

The candidate has been involved in various activities, including serving as an officer and chair in Alpha Sigma Phi from January 2006 to May 2010, where they held positions such as Vice President and Social Chair, and participated in the Livestrong Foundation's Bike-a-thon. Their skills encompass accounting, accounts payable, accounts receivable, bank reconciliations, cash flow projections, financial reporting, job costing, and proficiency in various software applications.


CV DATA:
{profile_record}
"""

    summaries = []

    for profile_record in tqdm(cv_list, desc="Resumiendo CVs"):
        prompt = prompt_base.format(profile_record=profile_record)

        response = await client.chat.completions.create(
            model=DEPLOYMENT,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperatura,
            max_tokens=1000

        )
        texto = response.choices[0].message.content

        # 1️⃣ Eliminar los backticks triple
        texto = texto.replace("```csv", "").replace("```", "")

        # 2️⃣ Limpiar espacios al inicio y final
        texto = texto.strip()

        # 3️⃣ Si empieza con "csv ", quítalo
        if texto.lower().startswith("csv "):
            texto = texto[4:]

        # 4️⃣ Limpiar espacios redundantes
        texto = " ".join(texto.split())
        summaries.append(texto)

    return summaries


In [4]:
async def resumir_JD(
    jd_list,
    overwrite=True,
    temperatura=0.4
):
      promt_jd = """
          You will receive a structured input containing the following fields:

              - Job Position

              - Education

              - Experience

              - Age

              - JD Responsibilities [list]

              - Required Skills [list]

          Your task is to generate a concise, fluent paragraph that summarizes the information provided.

          Rules to follow strictly:

            - Do not add, remove, or infer any information — only rephrase and reorganize what is explicitly given.

            - If any field is null, empty, or not provided, omit it entirely from the summary.

            - Ensure that all values in list fields (e.g., responsibilities, skills) are included exactly as they appear, with no alterations or exclusions.

            - Output a single, well-structured paragraph in formal English.

        Example format when all fields are present:
        "The position is for a [Job Position] requiring a minimum of [Experience] of experience. The candidate should have an educational background of [Education] and is [Age] years old. Key responsibilities include [list all JD Responsibilities in a sentence]. The required skills for this role include [list all Required Skills in a sentence]."

        Example when some fields are missing:
        "The position is for a [Job Position] requiring a minimum of [Experience] of experience. Key responsibilities include [...]. The required skills for this role include [...]."

        Now, generate the summary based strictly on the available input.

        JD DATA:
        {profile_record}
      """
      jd_desc = []

      for profile_record in tqdm(jd_list, desc="Resumiendo JDs"):
          prompt = promt_jd.format(profile_record=profile_record)

          response = await client.chat.completions.create(
              model=DEPLOYMENT,
              messages=[{"role": "user", "content": prompt}],
              temperature=temperatura,
              max_tokens=1000

          )
          texto = response.choices[0].message.content

          # 1️⃣ Eliminar los backticks triple
          texto = texto.replace("```csv", "").replace("```", "")

          # 2️⃣ Limpiar espacios al inicio y final
          texto = texto.strip()

          # 3️⃣ Si empieza con "csv ", quítalo
          if texto.lower().startswith("csv "):
              texto = texto[4:]

          # 4️⃣ Limpiar espacios redundantes
          texto = " ".join(texto.split())
          jd_desc.append(texto)

      return jd_desc



In [5]:
import sys

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.append('/content/drive/MyDrive/ApplAI/')

Mounted at /content/drive


In [6]:
import pandas as pd

def leer_cv_columna(ruta_archivo, nombre_columna="CV"):
    """
    Lee un archivo CSV y extrae los valores de la columna de CVs.

    Parámetros:
    - ruta_archivo (str): Ruta al archivo CSV (por ejemplo: 'data/cvs.csv')
    - nombre_columna (str): Nombre de la columna que contiene los CVs (default: 'CV')

    Retorna:
    - Lista de CVs (list[str])
    """
    df = pd.read_csv(ruta_archivo, encoding='utf-8')

    if nombre_columna not in df.columns:
        raise ValueError(f"La columna '{nombre_columna}' no existe en el archivo.")

    # Eliminar valores nulos y convertir a string
    cv_list = df[nombre_columna].dropna().astype(str).tolist()

    return cv_list

In [7]:
import pandas as pd
import os

def guardar_cv_con_jd_y_score(summaries, jd_list, score_list, indices_origen, ruta_salida, overwrite=True):
    if not overwrite:
        base, ext = os.path.splitext(ruta_salida)
        contador = 1
        while os.path.exists(ruta_salida):
            ruta_salida = f"{base}_{contador}{ext}"
            contador += 1

    # Subseleccionar los JD y score según los índices
    jd_seleccionados = [jd_list[i] for i in indices_origen]
    score_seleccionados = [score_list[i] for i in indices_origen]

    # Crear DataFrame
    df = pd.DataFrame({
        "cv_information": summaries,
        "job_description": jd_seleccionados,
        "score": score_seleccionados
    })

    # Guardar CSV
    df.to_csv(ruta_salida, index=False, encoding="utf-8")
    print(f"✅ Archivo combinado guardado en: {ruta_salida}")
    return ruta_salida


In [8]:
cv_list = leer_cv_columna("/content/drive/MyDrive/ApplAI/parte_4.csv")
JD_list = leer_cv_columna("/content/drive/MyDrive/ApplAI/parte_4.csv", "JD")
score_list = leer_cv_columna("/content/drive/MyDrive/ApplAI/parte_4.csv", "score")

summaries = await resumir_CSV(cv_list, overwrite=False)

Resumiendo CVs: 100%|██████████| 1363/1363 [51:25<00:00,  2.26s/it]


In [9]:
jd_summaries = await resumir_JD(JD_list, overwrite=False)

Resumiendo JDs: 100%|██████████| 1363/1363 [26:28<00:00,  1.17s/it]


In [10]:
guardar_cv_con_jd_y_score(
    summaries=summaries,
    jd_list=jd_summaries,
    score_list=score_list,
    indices_origen=list(range(0,len(summaries))),
    ruta_salida="/content/drive/MyDrive/ApplAI/parteCambiada_parte_4.csv",
    overwrite=False
)

✅ Archivo combinado guardado en: /content/drive/MyDrive/ApplAI/parteCambiada_parte_4.csv


'/content/drive/MyDrive/ApplAI/parteCambiada_parte_4.csv'

# Función que llama al LLM y parsea el resultado
async def extract_cv_info(texto_crudo: str, azure_client: AsyncAzureOpenAI, DEPLOYMENT: str):
    system_prompt = "You are an assistant that extracts structured information from CVs."

    # Prompt
    user_prompt = f"""
You will receive a resume in raw text format. Your task is to extract:

1. The name of the candidate.
2. The email address.
3. The phone number.
4. A cleaned and structured version of the resume that removes any personal contact information (name, email, phone, address, LinkedIn, etc.).

For the fourth part:
- Do NOT summarize or omit key content.
- Instead, preserve as much of the original job-related information as possible.
- Reorganize and rephrase disconnected items into full sentences with proper structure and connectors (e.g., “The candidate worked at...”, “They were responsible for...”, “Their skills include...”).
- You may rewrite bullet points and lists as prose, but keep all relevant details intact.
- Do NOT include any personal identifiers or contact information.
- Imagine you are preparing the resume for analysis by an AI model – you want to keep the full context but make it more readable.

You may use the following fields *only if present* in the text:
- Career Objective
- Skills
- Institution
- Degree
- Results
- Result Type
- Field of Study
- Companies
- Job Skills
- Positions
- Responsibilities
- Activity Type
- Organizations
- Roles
- Languages
- Proficiency
- Certifications From
- Certification Skills

Return your answer strictly in JSON format as follows:

{{
    "name": "...",
    "email": "...",
    "phone_number": "...",
    "cv_information": "..."
}}

CV TEXT:
{texto_crudo}
"""
    try:
        response = await azure_client.chat.completions.create(
            model=DEPLOYMENT,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0,
            max_tokens=1000
        )

        content = response.choices[0].message.content.strip()

        # Eliminar formato tipo markdown de código json ... 
        if content.startswith("json"):
            content = content.removeprefix("json").removesuffix("").strip()
        elif content.startswith(""):
            content = content.removeprefix("").removesuffix("").strip()

        # Intentar cargar JSON
        data = json.loads(content)

        # Rellenar claves faltantes manualmente
        data = {
                "name": data.get("name", "Not provided"),
                "email": data.get("email", "Not provided"),
                "phone_number": data.get("phone_number", "Not provided"),
                "cv_information": data.get("cv_information", "Not provided")
            }
        return CleanedCV(**data)

    except Exception as e:
        print("⚠️ Error al procesar la respuesta del modelo:", e)
        print("🔎 Respuesta recibida:")
        print(content)
        return CleanedCV()